In [26]:
# Define storage account and container
storage_account = "ds562team9datalake"
container_name = "silver"
file_path = "historical_crypto_tweet_data/historical_crypto_tweets_2014_2019.parquet"

storage_key = "KXg2Djg7uRevBSpPNIVnKw/N6HpqBh+kJwDX07wkywbpU2joMZdTIBOXk30EoMMxH2d8wwb+9j0g+AStO60IWw=="

spark.conf.set(f"fs.azure.account.key.{storage_account}.blob.core.windows.net", storage_key)
spark.conf.set("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
spark.conf.set(
    "spark.sql.sources.commitProtocolClass",
    "org.apache.spark.internal.io.cloud.PathOutputCommitProtocol"
)
spark.conf.set(
    "spark.sql.parquet.output.committer.class",
    "org.apache.spark.sql.execution.datasources.parquet.DirectParquetOutputCommitter"
)

url = f"wasbs://{container_name}@{storage_account}.blob.core.windows.net/{file_path}"

In [27]:
df = spark.read.parquet(url)
df.printSchema()
df.show(5, truncate=False)

In [28]:
from pyspark.sql.functions import col

# This filters rows where the 'text' column contains only ASCII characters
df_english = df.filter(col("text").rlike("^[\\x00-\\x7F]+$"))

# Now count
total_rows = df.count()
english_rows = df_english.count()

print(f"Total rows: {total_rows}")
print(f"English-like tweets (ASCII-only): {english_rows}")

In [29]:
df_english.show(5, truncate=False)
df_english.groupBy("year").count().orderBy("year").show()

In [30]:
spark.conf.set(
    "fs.azure.account.key.ds562team9datalake.blob.core.windows.net",
    "KXg2Djg7uRevBSpPNIVnKw/N6HpqBh+kJwDX07wkywbpU2joMZdTIBOXk30EoMMxH2d8wwb+9j0g+AStO60IWw=="
)

In [31]:
# Fix the committer issue
spark.conf.set("spark.sql.parquet.output.committer.class", "org.apache.parquet.hadoop.ParquetOutputCommitter")

In [32]:
# Then proceed to write
df_english.coalesce(1).write.mode("overwrite").parquet(
    "wasbs://bronze@ds562team9datalake.blob.core.windows.net/Historical_crypto_tweets/cleaned/"
)